In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.interpolate import interpolate

In [2]:
class Interpolation():
    
    @classmethod
    def intrp_outlier(self, df, feature_col, ind_col, method_no = 1):
        self.ind_col = ind_col # independent variable, the column that is used for interpolation
        self.feature_col = feature_col # column/variable that ouliers have to be corrected in
        # print('Choose a number of interpolation methods:', method_dict)
        method_dict = {1: 'linear', 2: 'nearest', 3: 'zero', 4: 'slinear', 5: 'quadratic', 6: 'cubic'}
        self.interp_method = method_dict[method_no] # the interpolation method
        self.df = df # original dataframe 
        self.df_corrected = pd.DataFrame({'A' : []}) # with values corrected once the interpolation function applied to it 
        self.f = None # the interpolation function
        self.outlier_col = None # column of zero and ones indicating ouliers as 1 and inliers as 0
        self.df_no_outlier_no_missing = None # dataframe with outliers and missing values removed
        self.df_only_outlier = None # dataframe with outliers only
        
        # We now detect outliers and add a column with 0 if it is not outlier and 1 if it is outlier        
        dfMedian= self.df[self.feature_col].quantile(0.50)
        lower_quartile= self.df[self.feature_col].quantile(0.25)
        upper_quartile= self.df[self.feature_col].quantile(0.75)
        iqr = upper_quartile - lower_quartile
        upper_whisker = self.df[self.feature_col][self.df[self.feature_col]<=upper_quartile+1.5*iqr].max()
        lower_whisker = self.df[self.feature_col][self.df[self.feature_col]>=lower_quartile-1.5*iqr].min()
        self.outlier_col = np.where((self.df[self.feature_col] > upper_whisker)|(self.df[self.feature_col] < lower_whisker), 1, 0) 
        self.df['outlier'] = self.outlier_col
        self.df_no_outlier_no_missing = self.df[self.outlier_col == 0].dropna()
        self.df_only_outlier = self.df[self.outlier_col == 1] 
        
        # We now replace the outliers detected with interpolation
        if self.interp_method in ['quadratic', 'cubic']:
            print('warning: for this interpolation algorithm, the independent variables should be stricktly ascending!')
            self.df_no_outlier_no_missing.sort_values(by=[self.ind_col], inplace=True)        
            self.f = interpolate.interp1d(np.array(self.df_no_outlier_no_missing[self.ind_col]), 
                                          np.array(self.df_no_outlier_no_missing[self.feature_col]),
                                          kind=self.interp_method, 
                                          bounds_error=False,
                                          #fill_value="extrapolate")
                                          fill_value=self.df_no_outlier_no_missing[self.feature_col].mean())
        else:
            self.f = interpolate.interp1d(np.array(self.df_no_outlier_no_missing[self.ind_col]), 
                                          np.array(self.df_no_outlier_no_missing[self.feature_col]),
                                          kind=self.interp_method, 
                                          bounds_error=False,
                                          # fill_value="extrapolate")
                                          fill_value=np.array(self.df_no_outlier_no_missing[self.feature_col]).mean())
        # This is the last part
        if self.df_corrected.empty == True: 
            self.df_corrected = self.df.copy(deep=True)
        self.df_corrected[self.feature_col][self.outlier_col == 1] = np.array(self.f(self.df_only_outlier[self.ind_col]))
        
        return self.df_corrected
    
    @classmethod
    def intrp_missing(self, df, feature_col, ind_col, method_no = 1):  
        self.ind_col = ind_col # independent variable, the column that is used for interpolation
        self.feature_col = feature_col # column/variable that ouliers have to be corrected in
        # print('Choose a number of interpolation methods:', method_dict)
        method_dict = {1: 'linear', 2: 'nearest', 3: 'zero', 4: 'slinear', 5: 'quadratic', 6: 'cubic'}
        self.interp_method = method_dict[method_no] # the interpolation method
        self.df = df # original dataframe 
        self.df_corrected = pd.DataFrame({'A' : []}) # with values corrected once the interpolation function applied to it 
        self.f = None # the interpolation function  
        # Interpolate if missing
        if self.interp_method in ['quadratic', 'cubic']:
            print('warning: for this interpolation algorithm, the independent variables should be stricktly ascending!')
            self.df_no_outlier_no_missing.sort_values(by=[self.ind_col], inplace=True)        
            self.f = interpolate.interp1d(np.array(self.df_no_outlier_no_missing[self.ind_col]), 
                                          np.array(self.df_no_outlier_no_missing[self.feature_col]),
                                          kind=self.interp_method, 
                                          bounds_error=False,
                                          #fill_value="extrapolate")
                                          fill_value=self.df_no_outlier_no_missing[self.feature_col].mean())
        else:
            self.f = interpolate.interp1d(np.array(self.df_no_outlier_no_missing[self.ind_col]), 
                                          np.array(self.df_no_outlier_no_missing[self.feature_col]),
                                          kind=self.interp_method, 
                                          bounds_error=False,
                                          # fill_value="extrapolate")
                                          fill_value=np.array(self.df_no_outlier_no_missing[self.feature_col]).mean())
        # Next Part    
        if self.df_corrected.empty == True: 
            self.df_corrected = self.df.copy(deep=True)
        self.df_corrected[self.feature_col][self.df[self.feature_col].isna()] = np.array(self.f(self.df[self.df[self.feature_col].isna()][self.ind_col]))
                
        return self.df_corrected

In [3]:
df = pd.read_excel('TimeSeries Data from biogas plant.xlsx')
df.describe() # Quick short description of the data set
print(df.columns)

Index(['Date and time [Matlab_Time)', 'Biogas [m³/h] Cogeneration unit I',
       'Biogas [m³/h] Cogeneration unit II',
       'Methane content in the gas CH4 [%]', 'Total Solids TS-Wert [%]',
       'pH-Value'],
      dtype='object')


In [4]:
df.head(10) # outlier column will be added with the interpolation class

,Date and time [Matlab_Time),Biogas [m³/h] Cogeneration unit I,Biogas [m³/h] Cogeneration unit II,Methane content in the gas CH4 [%],Total Solids TS-Wert [%],pH-Value
0,734124.999317,1262.000000,4790.000000,53.53009,7.442129,NaN
1,734125.082650,1222.583790,4802.499855,53.53009,7.959729,7.706535
2,734125.165984,1183.167579,4814.999711,53.53009,7.453511,7.691780
3,734125.249317,1143.751369,4827.499566,53.53009,8.026361,7.679975
4,734125.332650,1104.335158,4839.999421,53.53009,7.560807,7.702491
5,734125.415984,1064.918948,4852.499277,53.53009,7.569912,7.697105
6,734125.499317,1025.502737,4864.999132,53.53009,7.384810,7.696759
7,734125.582650,986.086527,4877.498987,53.53009,7.572945,7.697858
8,734125.665984,946.670316,4889.998843,53.53009,7.941835,7.699016
9,734125.749317,907.254106,4902.498698,53.53009,8.099210,7.708971


In [5]:
# create and interpolation object
feature_col = 'pH-Value'
ind_col = 'Date and time [Matlab_Time)'
method_no = 1 #  choose a number between 1 to 6, interpolation methods: {1: 'linear', 2: 'nearest', 3: 'zero', 4: 'slinear', 5: 'quadratic', 6: 'cubic'}

Interpolation.intrp_outlier(df, feature_col, ind_col, method_no)

<ipython-input-2-d1f7e8e3f4b0>:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.df_corrected[self.feature_col][self.outlier_col == 1] = np.array(self.f(self.df_only_outlier[self.ind_col]))


,Date and time [Matlab_Time),Biogas [m³/h] Cogeneration unit I,Biogas [m³/h] Cogeneration unit II,Methane content in the gas CH4 [%],Total Solids TS-Wert [%],pH-Value,outlier
0,734124.999317,1262.000000,4790.000000,53.530090,7.442129,NaN,0
1,734125.082650,1222.583790,4802.499855,53.530090,7.959729,7.706535,0
2,734125.165984,1183.167579,4814.999711,53.530090,7.453511,7.691780,0
3,734125.249317,1143.751369,4827.499566,53.530090,8.026361,7.679975,0
4,734125.332650,1104.335158,4839.999421,53.530090,7.560807,7.702491,0
...,...,...,...,...,...,...,...
1616,734259.665984,2510.665494,5129.999190,52.710275,8.032442,7.612783,0
1617,734259.749317,2535.999120,5147.499392,52.408863,8.414378,7.609568,0
1618,734259.832650,2561.332747,5164.999595,52.107451,8.796314,7.606353,0
1619,734259.915984,2586.666373,5182.499797,51.806039,9.178250,7.603138,0


In [6]:
Interpolation.intrp_missing(df, feature_col, ind_col, method_no)

<ipython-input-2-d1f7e8e3f4b0>:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.df_corrected[self.feature_col][self.df[self.feature_col].isna()] = np.array(self.f(self.df[self.df[self.feature_col].isna()][self.ind_col]))


,Date and time [Matlab_Time),Biogas [m³/h] Cogeneration unit I,Biogas [m³/h] Cogeneration unit II,Methane content in the gas CH4 [%],Total Solids TS-Wert [%],pH-Value,outlier
0,734124.999317,1262.000000,4790.000000,53.530090,7.442129,7.525463,0
1,734125.082650,1222.583790,4802.499855,53.530090,7.959729,7.706535,0
2,734125.165984,1183.167579,4814.999711,53.530090,7.453511,7.691780,0
3,734125.249317,1143.751369,4827.499566,53.530090,8.026361,7.679975,0
4,734125.332650,1104.335158,4839.999421,53.530090,7.560807,7.702491,0
...,...,...,...,...,...,...,...
1616,734259.665984,2510.665494,5129.999190,52.710275,8.032442,7.612783,0
1617,734259.749317,2535.999120,5147.499392,52.408863,8.414378,7.609568,0
1618,734259.832650,2561.332747,5164.999595,52.107451,8.796314,7.606353,0
1619,734259.915984,2586.666373,5182.499797,51.806039,9.178250,7.603138,0
